In [22]:
# https://chatgpt.com/c/672fd1fa-0a34-8012-bf01-13f9c22d4870?model=o1-preview
# https://chatgpt.com/c/673ba83e-09e4-8012-bfa1-5bcef163edc1

import os
import json
import pandas as pd

class DataProcessor:
  def read_files(self, directory: str):
    file_names = [f for f in os.listdir(directory) if f.endswith('.csv')]

    file_name = file_names[0]

    # for file_name in file_names:
    file_path = os.path.join(directory, file_name)
    data = pd.read_csv(file_path)

  def process(self):
    pass

In [ ]:
from google.colab import drive

drive.mount("/content/drive", force_remount=True)

In [23]:
BASE_DIRECTORY = "/content/drive/My Drive"
TARGET = ""

PATH = os.path.join(BASE_DIRECTORY, TARGET)

process = DataProcessor()
process.read_files(PATH)
process.process()